<a href="https://colab.research.google.com/github/hasibzunair/fifa-demo/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FIFA Demo Inference

This notebook generates a synthetic image of a person wearing a target clothing. It requires an image of a person and a target clothing as input.

### Setup dependencies

In [ ]:
!git clone https://github.com/hasibzunair/fifa-demo.git
%cd fifa-demo

In [ ]:
!pip install ninja

In [ ]:
import gdown
import numpy as np
from PIL import Image
import IPython
import gdown
import os
import sys

from predict_pose import generate_pose_keypoints

In [ ]:
# Create dummy folders same as train/test data dir tree

!mkdir Data_preprocessing/test_color
!mkdir Data_preprocessing/test_colormask
!mkdir Data_preprocessing/test_edge
!mkdir Data_preprocessing/test_img
!mkdir Data_preprocessing/test_label
!mkdir Data_preprocessing/test_mask
!mkdir Data_preprocessing/test_pose
!mkdir inputs
!mkdir inputs/img
!mkdir inputs/cloth

In [ ]:
# Get pose model

%cd pose
!wget https://github.com/hasibzunair/fifa-demo/releases/download/v1.0/pose_deploy_linevec.prototxt
!wget https://github.com/hasibzunair/fifa-demo/releases/download/v1.0/pose_iter_440000.caffemodel
%cd ..

In [ ]:
# Get parser and segmentation model

!git clone https://github.com/hasibzunair/Self-Correction-Human-Parsing-for-ACGPN.git
!git clone https://github.com/hasibzunair/U-2-Net.git

In [ ]:
# For segmentation mask generation

!wget https://github.com/hasibzunair/fifa-demo/releases/download/v1.0/lip_final.pth

In [ ]:
%cd U-2-Net
!mkdir saved_models
!mkdir saved_models/u2net
!mkdir saved_models/u2netp
!gdown --id 1rbSTGKAE-MTxBYHd-51l2hMOQPT_7EPy -O saved_models/u2netp/u2netp.pth
!gdown --id 1ao1ovG1Qtx4b7EoskHXmi2E9rp5CHLcZ -O saved_models/u2net/u2net.pth
import u2net_load
import u2net_run
u2net = u2net_load.model(model_name = 'u2netp')
%cd ..

In [ ]:
# Get model checkpoints

!mkdir checkpoints
%cd checkpoints
!wget https://github.com/hasibzunair/fifa-demo/releases/download/v1.0/final.zip
!unzip final.zip
%cd ..

### Please upload your cloth image below

Some assumptions: 
* PNG image files
* Name your cloth image as `cloth.png` and person image as `person.png`.

In [ ]:
%cd inputs/cloth
from google.colab import files
uploaded = files.upload()
%cd ..
%cd ..

### Please upload your person image below

In [ ]:
%cd inputs/img
from google.colab import files
uploaded = files.upload()
%cd ..
%cd ..

### Preprocessing

In [ ]:
cloth_name = 'cloth.png'
cloth_path = os.path.join('inputs/cloth', sorted(os.listdir('inputs/cloth'))[0])
cloth = Image.open(cloth_path)
# Resize cloth image
cloth = cloth.resize((192, 256), Image.BICUBIC).convert('RGB')

# Save resized cloth image
cloth.save(os.path.join('Data_preprocessing/test_color', cloth_name))

# 1. Get binary mask for clothing image
u2net_run.infer(u2net, 'Data_preprocessing/test_color', 'Data_preprocessing/test_edge')

In [ ]:
import time

start_time = time.time()
# Person image
img_name = 'person.png'
img_path = os.path.join('inputs/img', sorted(os.listdir('inputs/img'))[0])
img = Image.open(img_path)
img = img.resize((192, 256), Image.BICUBIC)
img_path = os.path.join('Data_preprocessing/test_img', img_name)
img.save(img_path)
resize_time = time.time()
print('Resized image in {}s'.format(resize_time-start_time))

# 2. Get parsed person image (test_label), uses person image
!python3 Self-Correction-Human-Parsing-for-ACGPN/simple_extractor.py --dataset 'lip' --model-restore 'lip_final.pth' --input-dir 'Data_preprocessing/test_img' --output-dir 'Data_preprocessing/test_label'
parse_time = time.time()
print('Parsing generated in {}s'.format(parse_time-resize_time))

# 3. Get pose map from person image
pose_path = os.path.join('Data_preprocessing/test_pose', img_name.replace('.png', '_keypoints.json'))
generate_pose_keypoints(img_path, pose_path)
pose_time = time.time()
print('Pose map generated in {}s'.format(pose_time-parse_time))

In [ ]:
!rm -rf Data_preprocessing/test_pairs.txt

# Format: person, cloth image
with open('Data_preprocessing/test_pairs.txt','w') as f:
    f.write('person.png cloth.png')

### Run inference

In [ ]:
# Run test.py using the preferred configuration (e.g. changes in architecture etc.)
!python test.py --name final

### Show results

In [ ]:
# See 'results' directory
output_grid = np.concatenate([np.array(Image.open('Data_preprocessing/test_img/person.png')),
                np.array(Image.open('Data_preprocessing/test_color/cloth.png')),
                np.array(Image.open('results/test/try-on/person.png'))], axis=1)
image_grid = Image.fromarray(output_grid)
image_grid

**To try a new person and cloth pair, remove current files and start again by uploading a new cloth image!**

In [ ]:
!rm -rf Data_preprocessing/test_color/*
!rm -rf Data_preprocessing/test_colormask/*
!rm -rf Data_preprocessing/test_edge/*
!rm -rf Data_preprocessing/test_img/*
!rm -rf Data_preprocessing/test_label/*
!rm -rf Data_preprocessing/test_mask/*
!rm -rf Data_preprocessing/test_pose/*
!rm -rf inputs/cloth/*
!rm -rf inputs/img/*
!rm -rf results/*
%cd /content/fifa-demo